In [ ]:
#importing libraries
import requests
from bs4 import BeautifulSoup as bs
from requests import get
import pandas as pd
from datetime import date
from datetime import datetime
import time

#create timestamp from string in "mm/dd/yyyy" format
def maketimestamp(s):
    if s == "start":
        return(1277960400)
    elif s == "today":
        return(int(time.mktime(date.today().timetuple())))
    #convert string into datetime format
    sdate = datetime.strptime(s, "%m/%d/%Y")

    #raise exception if date is outside range
    today = date.today()
    startdate = datetime.strptime("07/01/2010", "%m/%d/%Y")
    if sdate < startdate or sdate > today:
        raise Exception("Date must be between July 1, 2010, and Today")
    
    #convert datetime to timestamp and return
    timestamp = time.mktime(sdate.timetuple())
    return(int(timestamp))

#create incident search query URL from two dates
def makeurl(start, end):
    #convert dates to timestamps
    startdate = maketimestamp(start)
    enddate = maketimestamp(end)

    #switch start and end if in opposite order
    if startdate > enddate:
        (startdate, enddate) = (enddate, startdate)
    
    #create url from start and end dates and return
    url = "https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=" \
        + str(startdate) + "&endDate=" + str(enddate)
    return(url)

#find number of pages for a query URL
def findpagenum(url):
    #convert to beutifal soup and find div with page count
    page = get(url)
    soup = bs(page.content, 'html.parser')
    mydivs = soup.find_all("li", {"class": "page-count"})

    #convert page number html to a page count and return
    pagecount = str(mydivs[0].findChildren('span'))
    pagecount = int(pagecount.split("/")[1].split("<")[0])
    return(pagecount)
    
#scrape website for all reports for a query URL
def read(url):
    #iterate through all pages of incident query
    for i in range(0, findpagenum(url)):
        #change URL for each page
        offset = i * 5
        webpage =  url + "&offset=" + str(offset)
        
        #create dataframe for first page
        if offset == 0:
            df = pd.read_html(webpage)[0]

        #apend data from new pages to dataframe
        else:
            #get table data
            try:
                newdata = pd.read_html(webpage)[0]
                df = df.append(newdata)
            except:
                print("Error! Broken Page: " + website)
                
        #print statements for troubleshooting
        ##print(i)
        ##print(webpage)
    
    #reset index and return datafram
    df = df.reset_index().drop(["index"], axis=1)
    return(df)

#scrapes all UCPD incidents between two dates
#with no dates, will scrape between start date and today
#with one date, will scrape between start date and provided date
#with two dates, will scrape between provided dates 
#dates must be in "mm/dd/yyyy" format, but order of dates does not matter
def scrape(end = "today", start = "start"):
    url = makeurl(start, end)   
    df = read(url)
    return(df)

In [ ]:
df = scrape("start","start")

In [ ]:
df.to_csv("newoutput.csv")